In [23]:
from sklearn.model_selection import train_test_split
import json
import numpy as np

In [4]:
import random
###CREATING CLASSES TO MAKE CODE CLEANER###
##########################
#sentiment class just to assign the values
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

############################
#Review class assigns value to the review list => text, score and sentiment 
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        if self.score == 3:
            return Sentiment.NEUTRAL
        if self.score >=4:
            return Sentiment.POSITIVE
        
#############################
#this class balances the positive and negative reviews by shortning the positive reviews 
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
   
    def evenly_distribute(self):
        negative = list(filter(lambda x : x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x : x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[ :len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        
    def get_text(self): #assigns text value to the train_x
        return [x.text for x in self.reviews]
        
    def tk_sentiment(self): #assign sentiment value to the train_y
        return [x.sentiment for x in self.reviews]
        
        

In [5]:
#importing file and loading the neccesary data in the reviews list
import json
file_name = 'C:\\Users\\Admin\\Desktop\\datasets\\Books_small_10000.json'

reviews =[]
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall'])) 
        
reviews[5].sentiment

'POSITIVE'

# Prep data

In [6]:
#the train_test_split function
training, test = train_test_split(reviews, test_size= 0.33, random_state= 42)#separating dataset for traning and testing


In [7]:
#making the objects of the ReviewContainer class for training and testing
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [8]:
#running the function for training dataset
#distributing the reviews evenly so that our classifier is not biased towards on type of sentiment
train_container.evenly_distribute()
#separating review text and sentiment
train_x = train_container.get_text()
train_y = train_container.tk_sentiment()

#running the fuctions for testing dataset
test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.tk_sentiment()
####################################
#train_y
train_y.count(Sentiment.NEGATIVE)

436

# BAG of words vectorization

In [9]:
#converting the text of the reviews list to numbers throught CountVectorizer class 
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
#train_x_vectors[0].toarray()

test_x_vectors = vectorizer.transform(test_x)

#print(train_x[0])

# Support Vector Machine(SVM) Algorithm=>

In [10]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear') #using linear kernel
clf_svm.fit(train_x_vectors, train_y) #traning our svm model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
#reading a review
test_x[0]

"This first book in the Gabriella Giovanni series, and Kristi Belcamino's first publication, was a thrilling, edge of your seat thrill ride! You will NOT be disappointed. Cannot wait for the second book in the series, Blessed are the Meek."

In [12]:
#sentiment prediction of the above review by our classifier 
clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

# Decision Tree Algorithm =>

In [13]:
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
#training our model
clf_dec.fit(train_x_vectors, train_y)

#storing the predicted sentiments
pred_dec = clf_dec.predict(test_x_vectors)

In [14]:
pred_dec[: 10]

array(['NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE',
       'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE'],
      dtype='<U8')

In [15]:
test_y[: 10]

['POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE']

# Naive Bayes Algo=>

#ask Sir

In [25]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

# Logistic Regression Algorithm =>

In [17]:
from sklearn.linear_model import LogisticRegression

#making an object of logistic Regeression function
clf_log = LogisticRegression()
#traning our model
clf_log.fit(train_x_vectors, train_y)

#prediction
clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

# Evaluation

In [18]:
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
#print(clf_gnb.score(test_x_vectors, test_y))#ask sir first
print(clf_log.score(test_x_vectors, test_y))


0.7980769230769231
0.6658653846153846
0.8149038461538461


# F1 scores

In [19]:
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


array([0.82051282, 0.        , 0.808933  ])

In [20]:
#print the number of positive and negative reviews
#to check our model is not baised
print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


# Testing by giving some inputs

In [21]:
#Qualitative analysis
test_set = ['not satisfied', 'Nice book', 'Best book']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['NEGATIVE', 'POSITIVE', 'POSITIVE'], dtype='<U8')

# Taking user input and predicting

In [26]:
input_review = input("Enter your review: ")
input_review = [input_review]
vector_r = vectorizer.transform(input_review)
predict_r_svm = clf_svm.predict(vector_r)
predict_r_dec = clf_dec.predict(vector_r)
predict_r_log = clf_log.predict(vector_r)
print("Your review is : ")
print("by Support Vector Machine : "+ predict_r_svm[0])
print("by Decision Tree : "+ predict_r_dec[0])
print("by Logistic Regression : "+ predict_r_log[0])



Enter your review: bad
Your review is : 
by Support Vector Machine : NEGATIVE
by Decision Tree : POSITIVE
by Logistic Regression : NEGATIVE


In [ ]:
opt = 'y'
while opt=='y':
    input_review = input("Enter your review: ")
    input_review = [input_review]
    vector_r = vectorizer.transform(input_review)
    predict_r_svm = clf_svm.predict(vector_r)
    predict_r_dec = clf_dec.predict(vector_r)
    predict_r_log = clf_log.predict(vector_r)
    print("Your review is : ")
    print("by Support Vector Machine : "+ predict_r_svm[0])
    print("by Decision Tree : "+ predict_r_dec[0])
    print("by Logistic Regression : "+ predict_r_log[0])
    ch = string(input("Want to predict more (y/n) :"))
    opt = ch
